In [2]:
! pip install clip pinecone numpy pandas

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata (1.2 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/587.6 kB ? eta -:--:--
   --------------------------------------- 587.6/587.6 kB 15.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ------------ --------------------------- 3.


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     ------------------------------------- 536.2/536.2 kB 17.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/2.9 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.9 GB 23.5 MB/s eta 0:02:04
   ---------------------------------------- 0.0/2.9 GB 23.7 MB/s eta 0:02:03
   ---------------------------------------- 0.0/2.9 GB 23.6 MB/s eta 0:02:04
   ---------------------------------------- 0.0/2.9 GB 23.5 MB/s eta 0:02:04
   ---------------------------------------- 0.0/2.9 GB 22.4 MB/s eta 0:02:10
   -


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [60]:
import clip
import torch
from PIL import Image
import pinecone
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec

In [2]:
dataset = pd.read_csv('./Fashion Data/styles.csv',on_bad_lines='skip')

In [3]:
dataset.head(5)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [4]:
len(dataset)

44424

In [5]:
# Make sure there are atleast 5-10 samples from each combination of the three columns to ensure diversity

# Step 1: group by the three columns
grouped = dataset.groupby(["masterCategory", "subCategory", "articleType"])

# Step 2: sample 5–10 rows from each group (whichever is possible)
sampled = grouped.apply(lambda x: x.sample(min(len(x), 10), random_state=42)).reset_index(drop=True)

# Step 3: If sampled rows < 3000, fill up randomly until reaching 3000
if len(sampled) < 3000:
    remaining = dataset.drop(sampled.index).sample(3000 - len(sampled), random_state=42)
    final_df = pd.concat([sampled, remaining]).sample(frac=1, random_state=42).reset_index(drop=True)
else:
    final_df = sampled.sample(3000, random_state=42).reset_index(drop=True)

print(final_df.shape)  # should be (3000, number_of_columns)


(3000, 10)


C:\Users\jatta\AppData\Local\Temp\ipykernel_37984\2118823625.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled = grouped.apply(lambda x: x.sample(min(len(x), 10), random_state=42)).reset_index(drop=True)


In [6]:
final_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,3943,Women,Apparel,Topwear,Tshirts,Maroon,Summer,2011.0,Casual,Jealous 21 Women's Maroon T-shirt
1,49094,Women,Personal Care,Makeup,Kajal and Eyeliner,Purple,Spring,2017.0,Casual,Deborah Kajal 119
2,12118,Women,Apparel,Topwear,Kurtas,Brown,Fall,2011.0,Ethnic,Aurelia Women Solid Rust Brown Kurtas
3,48833,Women,Accessories,Jewellery,Pendant,Silver,Summer,2012.0,Casual,Lucera Women Silver Pendant
4,58494,Women,Apparel,Dress,Dresses,Purple,Fall,2012.0,Casual,Tonga Women Purple Dress
...,...,...,...,...,...,...,...,...,...,...
2995,8880,Men,Apparel,Bottomwear,Trousers,Brown,Summer,2011.0,Casual,Scullers Men Scul Brown Trousers
2996,57730,Women,Personal Care,Lips,Lip Care,Green,Spring,2017.0,Casual,Lotus Herbals Fresh Mint Lip Balm
2997,56509,Women,Personal Care,Lips,Lipstick,Pink,Spring,2017.0,Casual,Streetwear Lusicious Lips Hint of Pink Lipcolo...
2998,1550,Unisex,Sporting Goods,Sports Equipment,Footballs,White,Fall,2010.0,Sports,Puma Cat Trainer-WBL Football


In [7]:
print(len(final_df[final_df["gender"]=="Women"]))
print(len(final_df[final_df["gender"]=="Men"]))

1403
1242


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

100%|███████████████████████████████████████| 338M/338M [00:13<00:00, 25.5MiB/s]


In [9]:
images_path='./Fashion Data/images/'

In [33]:
Image_features = {}

for i in range(len(final_df)):
    img_name = str(final_df["id"][i])
    complete_image_path = os.path.join(images_path, img_name + '.jpg')
    try:
        image = preprocess(Image.open(complete_image_path)).unsqueeze(0).to(device)
    except Exception as e:
        print(f"Error processing image {img_name}: {e}")
        continue
    with torch.no_grad():
        features = model.encode_image(image).cpu().numpy()
    Image_features.update({int(final_df["id"][i]):features.flatten().tolist()})
    if i % 500 == 0:
        print(f"Processed {i} images")

Processed 0 images
Processed 500 images
Processed 1000 images
Processed 1500 images
Processed 2000 images
Error processing image 12347: [Errno 2] No such file or directory: './Fashion Data/images/12347.jpg'
Processed 2500 images


In [35]:
final_df["ImageFeatures"] = final_df["id"].map(Image_features)

In [36]:
final_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,ImageFeatures
0,3943,Women,Apparel,Topwear,Tshirts,Maroon,Summer,2011.0,Casual,Jealous 21 Women's Maroon T-shirt,"[0.038177490234375, 0.251953125, -0.4448242187..."
1,49094,Women,Personal Care,Makeup,Kajal and Eyeliner,Purple,Spring,2017.0,Casual,Deborah Kajal 119,"[-0.2347412109375, 0.06829833984375, 0.0147781..."
2,12118,Women,Apparel,Topwear,Kurtas,Brown,Fall,2011.0,Ethnic,Aurelia Women Solid Rust Brown Kurtas,"[-0.11053466796875, 0.2269287109375, 0.0972290..."
3,48833,Women,Accessories,Jewellery,Pendant,Silver,Summer,2012.0,Casual,Lucera Women Silver Pendant,"[-0.289794921875, -0.29345703125, 0.1068725585..."
4,58494,Women,Apparel,Dress,Dresses,Purple,Fall,2012.0,Casual,Tonga Women Purple Dress,"[-0.245849609375, 0.359130859375, -0.075683593..."
...,...,...,...,...,...,...,...,...,...,...,...
2995,8880,Men,Apparel,Bottomwear,Trousers,Brown,Summer,2011.0,Casual,Scullers Men Scul Brown Trousers,"[-0.051513671875, 0.1485595703125, -0.05270385..."
2996,57730,Women,Personal Care,Lips,Lip Care,Green,Spring,2017.0,Casual,Lotus Herbals Fresh Mint Lip Balm,"[0.06976318359375, -0.129638671875, -0.0204467..."
2997,56509,Women,Personal Care,Lips,Lipstick,Pink,Spring,2017.0,Casual,Streetwear Lusicious Lips Hint of Pink Lipcolo...,"[-0.280029296875, 0.055145263671875, -0.074768..."
2998,1550,Unisex,Sporting Goods,Sports Equipment,Footballs,White,Fall,2010.0,Sports,Puma Cat Trainer-WBL Football,"[0.1507568359375, -0.272216796875, -0.44677734..."


In [44]:
final_df = final_df.dropna(subset=["ImageFeatures"]).reset_index(drop=True)


In [45]:
final_df.isna().sum()

id                     0
gender                 0
masterCategory         0
subCategory            0
articleType            0
baseColour             1
season                 1
year                   0
usage                 78
productDisplayName     1
ImageFeatures          0
dtype: int64

In [47]:
final_df


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,ImageFeatures
0,3943,Women,Apparel,Topwear,Tshirts,Maroon,Summer,2011.0,Casual,Jealous 21 Women's Maroon T-shirt,"[0.038177490234375, 0.251953125, -0.4448242187..."
1,49094,Women,Personal Care,Makeup,Kajal and Eyeliner,Purple,Spring,2017.0,Casual,Deborah Kajal 119,"[-0.2347412109375, 0.06829833984375, 0.0147781..."
2,12118,Women,Apparel,Topwear,Kurtas,Brown,Fall,2011.0,Ethnic,Aurelia Women Solid Rust Brown Kurtas,"[-0.11053466796875, 0.2269287109375, 0.0972290..."
3,48833,Women,Accessories,Jewellery,Pendant,Silver,Summer,2012.0,Casual,Lucera Women Silver Pendant,"[-0.289794921875, -0.29345703125, 0.1068725585..."
4,58494,Women,Apparel,Dress,Dresses,Purple,Fall,2012.0,Casual,Tonga Women Purple Dress,"[-0.245849609375, 0.359130859375, -0.075683593..."
...,...,...,...,...,...,...,...,...,...,...,...
2994,8880,Men,Apparel,Bottomwear,Trousers,Brown,Summer,2011.0,Casual,Scullers Men Scul Brown Trousers,"[-0.051513671875, 0.1485595703125, -0.05270385..."
2995,57730,Women,Personal Care,Lips,Lip Care,Green,Spring,2017.0,Casual,Lotus Herbals Fresh Mint Lip Balm,"[0.06976318359375, -0.129638671875, -0.0204467..."
2996,56509,Women,Personal Care,Lips,Lipstick,Pink,Spring,2017.0,Casual,Streetwear Lusicious Lips Hint of Pink Lipcolo...,"[-0.280029296875, 0.055145263671875, -0.074768..."
2997,1550,Unisex,Sporting Goods,Sports Equipment,Footballs,White,Fall,2010.0,Sports,Puma Cat Trainer-WBL Football,"[0.1507568359375, -0.272216796875, -0.44677734..."


In [48]:
final_df.to_csv('sampled_fashion_data.csv', index=False)

In [64]:
len(final_df["ImageFeatures"][10])

512

In [68]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENVIRONMENT = "us-east-1"  # e.g., "us-west1-gcp"
INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")

In [57]:
load_dotenv()
pinecone_client = Pinecone(api_key=os.getenv("PINECONE_API_KEY")) 

In [61]:
def create_new_pinecone_index(index_name):
    try:  
        spec = ServerlessSpec(cloud='aws', region='us-east-1')    

        pinecone_client.create_index(  
            index_name,  
            dimension=512,  # dimensionality of text-embedding-ada-002  
            metric='dotproduct',  
            spec=spec  
        )
        
    except Exception as e:
        print(f"Error in init_pinecone: {e}")

In [62]:
create_new_pinecone_index("fashion-data")

In [74]:
pinecone_client = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pinecone_client.Index(os.getenv("PINECONE_INDEX_NAME"))

In [72]:
import os
import pandas as pd
from pinecone import Pinecone
import ast
from tqdm import tqdm

def upload_df_to_pinecone(df, batch_size=100):
    """
    Upload DataFrame to Pinecone with ImageFeatures as vectors and other columns as metadata.
    
    Args:
        df (pd.DataFrame): DataFrame containing the data
        index_name (str): Name of the Pinecone index
        batch_size (int): Number of records to upload in each batch (default: 100)
    
    Returns:
        dict: Summary of the upload process
    """
    
    # Initialize Pinecone client
    pinecone_client = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
    
    # Get the index
    index_name = os.getenv("PINECONE_INDEX_NAME")
    try:
        index = pinecone_client.Index(index_name)
    except Exception as e:
        print(f"Error accessing index '{index_name}': {e}")
        return {"success": False, "error": str(e)}
    
    # Prepare data for upload
    vectors_to_upsert = []
    
    for idx, row in df.iterrows():
        try:
            # Extract vector from ImageFeatures column
            if isinstance(row['ImageFeatures'], str):
                # If it's a string representation of a list, parse it
                vector = ast.literal_eval(row['ImageFeatures'])
            else:
                # If it's already a list/array
                vector = row['ImageFeatures']
            
            # Convert to list if it's a numpy array
            if hasattr(vector, 'tolist'):
                vector = vector.tolist()
            
            # Create metadata dictionary with all other columns
            metadata = {
                "gender": row["gender"] if pd.notna(row["gender"]) else "",
                "masterCategory": row["masterCategory"] if pd.notna(row["masterCategory"]) else "",
                "subCategory": row["subCategory"] if pd.notna(row["subCategory"]) else "",
                "articleType": row["articleType"] if pd.notna(row["articleType"]) else "",
                "baseColour": row["baseColour"] if pd.notna(row["baseColour"]) else "",
                "season": row["season"] if pd.notna(row["season"]) else "",
                "year": int(row["year"]) if pd.notna(row["year"]) else None,
                "usage": row["usage"] if pd.notna(row["usage"]) else "",
                "displayName": row["productDisplayName"] if pd.notna(row["productDisplayName"]) else "",
            }
            # for col in df.columns:
            #     if col != 'ImageFeatures':
            #         # Convert values to appropriate types for Pinecone metadata
            #         value = row[col]
            #         if pd.isna(value):
            #             metadata[col] = None
            #         elif isinstance(value, (int, float, str, bool)):
            #             metadata[col] = value
            #         else:
            #             # Convert other types to string
            #             metadata[col] = str(value)
            
            # Create vector entry
            vector_entry = {
                "id": str(row['id']),  # Use the 'id' column as the vector ID
                "values": vector,
                "metadata": metadata
            }
            
            vectors_to_upsert.append(vector_entry)
            
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            continue
    
    # Upload in batches
    total_vectors = len(vectors_to_upsert)
    successful_uploads = 0
    failed_uploads = 0
    
    print(f"Uploading {total_vectors} vectors to Pinecone index '{index_name}'...")
    
    for i in tqdm(range(0, total_vectors, batch_size)):
        batch = vectors_to_upsert[i:i + batch_size]
        
        try:
            # Upsert the batch
            upsert_response = index.upsert(vectors=batch)
            successful_uploads += len(batch)
            
        except Exception as e:
            print(f"Error uploading batch {i//batch_size + 1}: {e}")
            failed_uploads += len(batch)
    
    # Return summary
    summary = {
        "success": True,
        "total_vectors": total_vectors,
        "successful_uploads": successful_uploads,
        "failed_uploads": failed_uploads,
        "index_name": index_name
    }
    
    print(f"Upload completed: {successful_uploads} successful, {failed_uploads} failed")
    
    return summary

# Example usage:
# summary = upload_df_to_pinecone(final_df, "your-index-name")
# print(summary)

In [73]:
summary = upload_df_to_pinecone(final_df)

Uploading 2999 vectors to Pinecone index 'fashion-data'...


100%|██████████| 30/30 [00:12<00:00,  2.50it/s]

Upload completed: 2999 successful, 0 failed


In [79]:
def search_similar(query_image_path, top_k=5):
    # Process query image using the same preprocess function
    image = preprocess(Image.open(query_image_path)).unsqueeze(0).to(device)
    
    with torch.no_grad():
        query_features = model.encode_image(image).cpu().numpy()
    
    # Search in Pinecone with include_metadata=True
    results = index.query(
        vector=query_features.flatten().tolist(), 
        top_k=top_k,
        include_metadata=True  # This ensures metadata is returned
    )
    return results

In [76]:
from PIL import Image

In [77]:
img = Image.open("./Fashion Data/images/1163.jpg")
img.show()

In [88]:
results = search_similar("./Fashion Data/images/1525.jpg", top_k=5)

In [ ]:
for i in results['matches']:
    print(i["id"])

1697
20578
51330
15669
42938


: 

In [2]:
import pandas as pd

In [3]:
final_df = pd.read_csv('sampled_fashion_data.csv')
final_df = final_df.drop('ImageFeatures', axis=1)

In [4]:
final_df.head(4)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,3943,Women,Apparel,Topwear,Tshirts,Maroon,Summer,2011.0,Casual,Jealous 21 Women's Maroon T-shirt
1,49094,Women,Personal Care,Makeup,Kajal and Eyeliner,Purple,Spring,2017.0,Casual,Deborah Kajal 119
2,12118,Women,Apparel,Topwear,Kurtas,Brown,Fall,2011.0,Ethnic,Aurelia Women Solid Rust Brown Kurtas
3,48833,Women,Accessories,Jewellery,Pendant,Silver,Summer,2012.0,Casual,Lucera Women Silver Pendant


In [5]:
import pandas as pd
import base64
import os

# Example: your dataframe
# final_df = pd.read_csv("your_file.csv")

# Folder where images are stored
image_path = "./Fashion Data/images/"  # make sure it ends with /

# Function to convert image to base64
def image_to_base64(img_id):
    file_path = os.path.join(image_path, f"{str(img_id)}.jpg")
    if os.path.exists(file_path):
        with open(file_path, "rb") as img_file:
            return base64.b64encode(img_file.read()).decode("utf-8")
    else:
        return None  # handle missing images

# Apply function to create new column
final_df["image"] = final_df["id"].apply(image_to_base64)

# Preview
print(final_df[["id", "image"]].head())


      id                                              image
0   3943  /9j/4AAQSkZJRgABAQAAZABkAAD/7AARRHVja3kAAQAEAA...
1  49094  /9j/4AAQSkZJRgABAQEASABIAAD/7RqkUGhvdG9zaG9wID...
2  12118  /9j/4AAQSkZJRgABAQAAZABkAAD/7AARRHVja3kAAQAEAA...
3  48833  /9j/4AAQSkZJRgABAQEASABIAAD/7RJmUGhvdG9zaG9wID...
4  58494  /9j/4AAQSkZJRgABAQEASABIAAD/7RuGUGhvdG9zaG9wID...


In [7]:
final_df

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,3943,Women,Apparel,Topwear,Tshirts,Maroon,Summer,2011.0,Casual,Jealous 21 Women's Maroon T-shirt,/9j/4AAQSkZJRgABAQAAZABkAAD/7AARRHVja3kAAQAEAA...
1,49094,Women,Personal Care,Makeup,Kajal and Eyeliner,Purple,Spring,2017.0,Casual,Deborah Kajal 119,/9j/4AAQSkZJRgABAQEASABIAAD/7RqkUGhvdG9zaG9wID...
2,12118,Women,Apparel,Topwear,Kurtas,Brown,Fall,2011.0,Ethnic,Aurelia Women Solid Rust Brown Kurtas,/9j/4AAQSkZJRgABAQAAZABkAAD/7AARRHVja3kAAQAEAA...
3,48833,Women,Accessories,Jewellery,Pendant,Silver,Summer,2012.0,Casual,Lucera Women Silver Pendant,/9j/4AAQSkZJRgABAQEASABIAAD/7RJmUGhvdG9zaG9wID...
4,58494,Women,Apparel,Dress,Dresses,Purple,Fall,2012.0,Casual,Tonga Women Purple Dress,/9j/4AAQSkZJRgABAQEASABIAAD/7RuGUGhvdG9zaG9wID...
...,...,...,...,...,...,...,...,...,...,...,...
2994,8880,Men,Apparel,Bottomwear,Trousers,Brown,Summer,2011.0,Casual,Scullers Men Scul Brown Trousers,/9j/4AAQSkZJRgABAQAAZABkAAD/7AARRHVja3kAAQAEAA...
2995,57730,Women,Personal Care,Lips,Lip Care,Green,Spring,2017.0,Casual,Lotus Herbals Fresh Mint Lip Balm,/9j/4AAQSkZJRgABAQEASABIAAD/7REuUGhvdG9zaG9wID...
2996,56509,Women,Personal Care,Lips,Lipstick,Pink,Spring,2017.0,Casual,Streetwear Lusicious Lips Hint of Pink Lipcolo...,/9j/4AAQSkZJRgABAQEASABIAAD/7R3aUGhvdG9zaG9wID...
2997,1550,Unisex,Sporting Goods,Sports Equipment,Footballs,White,Fall,2010.0,Sports,Puma Cat Trainer-WBL Football,/9j/4AAQSkZJRgABAQAAAQABAAD//gA+Q1JFQVRPUjogZ2...


In [8]:
# --- Firestore uploader for a pandas DataFrame ---
# pip install firebase-admin

from typing import Optional
import os
import pandas as pd
import numpy as np
import firebase_admin
from firebase_admin import credentials, firestore


# 1) Initialize Firebase (call once)
def init_firebase(service_account_path: str) -> firestore.Client:
    """
    Initialize Firebase Admin SDK and return a Firestore client.
    """
    if not firebase_admin._apps:
        cred = credentials.Certificate(service_account_path)
        firebase_admin.initialize_app(cred)
    return firestore.client()


# 2) Helpers to sanitize row values
def _to_native(v):
    """Convert numpy scalars to native Python types so Firestore is happy."""
    if hasattr(v, "item"):
        try:
            return v.item()
        except Exception:
            return v
    return v

def _is_empty(v) -> bool:
    """Treat None/NaN/blank-string as empty."""
    if v is None:
        return True
    if isinstance(v, float) and np.isnan(v):
        return True
    if isinstance(v, str) and v.strip() == "":
        return True
    return False

def sanitize_row(row: pd.Series) -> dict:
    """
    Convert a DataFrame row into a Firestore-friendly dict:
    - None/NaN/blank -> ""
    - numpy types -> native Python types
    """
    payload = {}
    for col, val in row.items():
        if _is_empty(val):
            payload[col] = ""
        else:
            payload[col] = _to_native(val)
    return payload


# 3) Batch uploader
def upload_df_to_firestore(
    df: pd.DataFrame,
    collection_name: str,
    service_account_path: Optional[str] = None,
    client: Optional[firestore.Client] = None,
    batch_size: int = 450,  # Firestore limit is 500; keep headroom
):
    """
    Upload a DataFrame to Firestore.
    - Uses df['id'] (converted to str) as the document ID.
    - Writes rows in batches.
    - Empty values become "".
    """
    if "id" not in df.columns:
        raise ValueError("DataFrame must contain an 'id' column to be used as document ID.")

    # Create the client if not provided
    if client is None:
        # Allow using env var if path not passed explicitly
        if service_account_path is None:
            service_account_path = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS")
        if not service_account_path or not os.path.exists(service_account_path):
            raise FileNotFoundError(
                "Provide service_account_path or set GOOGLE_APPLICATION_CREDENTIALS env var."
            )
        client = init_firebase(service_account_path)

    col_ref = client.collection(collection_name)

    batch = client.batch()
    ops_in_batch = 0

    for _, row in df.iterrows():
        # Use 'id' column as document ID (string)
        doc_id = str(row["id"]) if not _is_empty(row["id"]) else ""
        if not doc_id:
            # skip rows without a usable id
            continue

        data = sanitize_row(row)
        # Optional: you can drop 'id' from the payload since it's already the doc ID
        # del data["id"]

        doc_ref = col_ref.document(doc_id)
        batch.set(doc_ref, data, merge=True)  # merge so re-runs are idempotent
        ops_in_batch += 1

        if ops_in_batch >= batch_size:
            batch.commit()
            batch = client.batch()
            ops_in_batch = 0

    # Commit any remaining writes
    if ops_in_batch:
        batch.commit()


# 4) Convenience getter (search by ID)
def get_doc_by_id(collection_name: str, doc_id: str, client: firestore.Client) -> dict:
    """
    Fetch a single document by its Firestore doc ID (which we set to the row's 'id').
    Returns {} if not found.
    """
    snap = client.collection(collection_name).document(str(doc_id)).get()
    return snap.to_dict() or {}


# -------- Example usage --------
# service_account_json = "/path/to/your-service-account.json"
# db = init_firebase(service_account_json)
# upload_df_to_firestore(final_df, collection_name="products", client=db)
# one = get_doc_by_id("products", "3943", db)
# print(one)


In [9]:
service_account_json = "./affinitylabs-b09b3-firebase-adminsdk-fbsvc-1d8e0bee01.json"
db = init_firebase(service_account_json)

In [10]:
upload_df_to_firestore(final_df, collection_name="FashionData", client=db)

In [11]:
one = get_doc_by_id("FashionData", "3943", db)

In [12]:
print(one)

{'articleType': 'Tshirts', 'masterCategory': 'Apparel', 'image': '/9j/4AAQSkZJRgABAQAAZABkAAD/7AARRHVja3kAAQAEAAAAQAAA/9sAQwAEAwMDAwIEAwMDBAQEBQYKBgYFBQYMCAkHCg4MDw4ODA0NDxEWEw8QFRENDRMaExUXGBkZGQ8SGx0bGB0WGBkY/9sAQwEEBAQGBQYLBgYLGBANEBgYGBgYGBgYGBgYGBgYGBgYGBgYGBgYGBgYGBgYGBgYGBgYGBgYGBgYGBgYGBgYGBgY/8AAEQgAUAA8AwERAAIRAQMRAf/EAB0AAAICAgMBAAAAAAAAAAAAAAAIBgkEBwECAwX/xAA2EAABAgUDAgQEAwgDAAAAAAABAgMABAUGERIhMQcIEyJBURQyYYGCkaEjM0JDUnFzwZKx8P/EABsBAAIDAQEBAAAAAAAAAAAAAAAGAwQFAQIH/8QAMxEAAQMCAwQJAwQDAAAAAAAAAQACAwQRBRIhMVFhoQYTFEFxgZGx0SLB8CMkMkIz4fH/2gAMAwEAAhEDEQA/AH+gQiBCCcCBCTmqd1HVe7us9ZtvoxZVIqFKozrss49VdaVPrQVIKyoKSEJKwdKdyQnJxnAzqmvZT/zNr7Fo02HvnBy9ymXQ7uWrV29Q5nph1Ut1i3rsaJSyqXyGZhQTqLZBUrSsp8ySCUqGeCBmWmq2y2sdDsUdRRuhvfu2plIuKkiBCIELo660wyp55xDbaBlS1kAAe5JjjnBoudi61pcbBROo9Q6PLBYkUqndPLoPhtD8Z5+wMYNV0ipobiP6jw2evwtqmwGol1f9I47fT5Ufnb1rsxQZutJlHmqVKMrmHplv9i2EIBUSFr3VsP4RFLtmI1jS5jcrN+zmdfQK52OhpXZXuzO3beQ09Uq3blTZ1y3axe9Hn5amNz9YmJ9yUbb8QqY1kIa1Z5HmyDnmKmITtEnVkXc0DVaVFBmZn/q4nQ